In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import cvae_idec.IDEC
tsne_0 = np.load('./TSNE_images/0.npy')
tsne_5 = np.load('./TSNE_images/5.npy')
tsne_10 = np.load('./TSNE_images/10.npy')
tsne_final = np.load('./TSNE_images/final.npy')

fig, axs = plt.subplots(2, 2, figsize=(10, 8))
axs[0,0].scatter(tsne_0[:,0], tsne_0[:,1])
axs[0,1].scatter(tsne_5[:,0], tsne_5[:,1])
axs[1,0].scatter(tsne_10[:,0], tsne_10[:,1])
axs[1,1].scatter(tsne_final[:,0], tsne_final[:,1])

ModuleNotFoundError: No module named 'cv2'

In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.optim import Adam
from torch.utils.data import DataLoader

from ae.vae_trainer import VAETrainer
#from ae.cvae_glau import CVAE
from ae.cvae_cifar import CVAE
#from ae.cvae import CVAE
from ae.vae import VAE
from torch.utils.data import Dataset


class IDEC(nn.Module):

    def __init__(self,
                 n_input,
                 n_z,
                 n_clusters,
                 alpha=1,
                 pretrain_path='saved_models/VAE/cvae_cifar10.pkl'):
        super(IDEC, self).__init__()
        self.alpha = 1.0
        self.pretrain_path = pretrain_path

        self.ae = CVAE(in_shape=n_input, n_z=n_z)
        # cluster layer
        self.cluster_layer = Parameter(torch.Tensor(n_clusters, n_z))
        torch.nn.init.xavier_normal_(self.cluster_layer.data)
        
    ### Pretrain AE #################################################################
    def pretrain(self, path=''):
        if path == '':
            pretrain_ae(self.ae)
        # load pretrain weights
        else:
            self.ae.load_state_dict(torch.load(self.pretrain_path))
            print('load pretrained ae from', path)

    def forward(self, x):
        x_bar, z, _, _ = self.ae(x)     
        # cluster
        q = 1.0 / (1.0 + torch.sum(
            torch.pow(z.unsqueeze(1) - self.cluster_layer, 2), 2) / self.alpha)
        q = q.pow((self.alpha + 1.0) / 2.0)
        q = (q.t() / torch.sum(q, 1)).t()

        return x_bar, q
    
def load_brain(path='../brain_data.npz'):
    f = np.load(path)
    x, y = f['x_train'], f['y_train']
    f.close()
    
    x = x.astype(np.float32)
    x = np.transpose(x.data, (0, 1, 3, 2))
    x = np.divide(x, 255.)
    y = y.astype(np.int32)
    print('Brain samples', x.shape, y.shape)
    return x, y
    
class BrainDataset(Dataset):

    def __init__(self):
        self.x, self.y = load_brain()

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, idx):
        return torch.from_numpy(np.array(self.x[idx])), torch.from_numpy(
            np.array(self.y[idx])), torch.from_numpy(np.array(idx))


In [45]:
n_clusters = 2
n_z = 2
batch_size = 16
dataset = BrainDataset()
n_input = dataset.x[0].shape

model = IDEC(
    n_input=n_input,
    n_z=n_z,
    n_clusters=n_clusters,
    alpha=1.0)

with open(".pkl", "rb") as fp:
    newmodel.load_state_dict(pickle.load(fp))


Brain samples (2251, 4, 128, 192) (2251, 192, 128)
